In [1]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.pyplot as plt
from scipy.ndimage import gaussian_filter
from scipy.signal import hamming
import numpy as np
from scipy import  ndimage
import os

def lattice(parms,nxx=20, nyy=20):
  a1 = parms[0]
  a2 = parms[1]
  phi = parms[2]
  
  nx,ny = np.meshgrid(np.arange(nxx), np.arange(nyy))
  values = (nx.ravel(),ny.ravel())
  values = np.array(values).T
  nxx = values[:,0]
  nyy = values[:,1]
  j = a1.shape[0]
  print(j)
  atom_pos = np.zeros((j,400,2))
  for i in range(0,j):

      x_ind = nxx * a1[i] + nyy * a2[i] * np.cos(phi[i])
      y_ind = nyy * a2[i] * np.sin(phi[i])
      atom_pos[i,:,0] = x_ind
      atom_pos[i,:,1] = y_ind
  
  return atom_pos


"""This function distorts the positions of all the atoms in the lattice with
standard deviation 30% of the lattice parameter a1"""
def distortions(atom_pos, parms):
  j = atom_pos.shape[0]
  a1 = parms[0]
  a2 = parms[1]
  atom_pos_dis = np.zeros((j,400,2))
  for i in range(j):
    x_dis = np.random.normal(loc = 0.0, scale = 0.03*a1[i] ,size = [X,1])
    y_dis = np.random.normal(loc = 0.0, scale = 0.03*a2[i] ,size = [X,1])

    dis = np.concatenate((x_dis,y_dis),axis=1)
    atom_pos_dis[i] = atom_pos[i] + dis
  
  return atom_pos_dis


def atom_to_image(atom_pos_dis,img_dim = 1024):
  j = atom_pos_dis.shape[0]
  image_atoms = np.zeros([j,img_dim,img_dim])
    
  for i in range(0,j):
      max_x = np.max(atom_pos_dis[i,:,0])
      max_y = np.max(atom_pos_dis[i,:,1])

      min_x = np.min(atom_pos_dis[i,:,0])
      min_y = np.min(atom_pos_dis[i,:,1])



      x1,y1 = atom_pos_dis[i,:,0], atom_pos_dis[i,:,1]
      x_img = ((x1 - min_x)/(max_x - min_x) * (img_dim-1))       
      y_img = ((y1-min_y)/(max_y - min_y) * (img_dim-1))

      x_img = x_img.astype(int)
      y_img = y_img.astype(int)  
      image_atoms[i,x_img, y_img]=1E6           


  return image_atoms

def convolve_atomic_image(image_atoms, sigma = 6):
  j = image_atoms.shape[0]
  con_img = np.zeros([j,1024,1024])
  for i in range(0,j):
      con_img[i] = gaussian_filter(image_atoms[i],sigma,order = 0)
  return con_img

def crop(convolved_img):
    crop_img = convolved_img[:,350:700,350:700]
    return crop_img

def fft(crop_img):
    img_ffts=[]
    fft_win_size =64
    j = crop_img.shape[0]
    for j in range(0,j):
        
        n = crop_img.shape[1]
        h = hamming(n) 
        ham2d = np.sqrt(np.outer(h,h))
        img_windowed = np.copy(crop_img[j])
        img_windowed *= ham2d 
        img_fft = np.fft.fftshift(np.fft.fft2(img_windowed))
        img_fft = img_fft[crop_img.shape[1]//2 - fft_win_size:crop_img.shape[1]//2+fft_win_size,
                                     crop_img.shape[1]//2 - fft_win_size:crop_img.shape[1]//2+fft_win_size]
        img_ffts.append(img_fft)
        
    return np.array(np.sqrt(np.abs(img_ffts)))
  
  
  


In [2]:
"""Debugged"""
import numpy as np
from sklearn.utils import shuffle

"""Generates 400 lattice parameters for each type of bravis lattice,
labels and shuffles them and output 100 data points when called"""

#oblique   a1 != a2, phi!=90 Label = 0
def oblique(X):
    bond_len = np.random.uniform(low = [0.8,0.8], high = [2.0,2.2],size = [X*2,2])
    diff = np.abs(((bond_len[:,0]-bond_len[:,1])*100)/bond_len[:,1])

    bond_len = bond_len[diff > 20.0,:]
    bond_len = bond_len[0:X,:]
    phi = np.zeros((X,1))
    phi[0:int(0.7*X)] = np.random.uniform(low = [0.0], high = [((55.0/180)*np.pi)],size = [int(0.7*X),1])
    phi[int(0.7*X):X] = np.random.uniform(low = [((65.0/180)*np.pi)], high = [((90.0/180)*np.pi)],size = [int(0.3*X),1])

    parms_obl = np.concatenate((bond_len,phi),axis = 1)
    return parms_obl.T
def square(X):
    #square  a1 = a2, phi = 90, Label = 1
    parms_sq = np.zeros([X,3])
    parms_sq[:,0] = np.random.uniform(low = 0.8, high = 2.0,size = X)
    parms_sq[:,1] = parms_sq[:,0]
    parms_sq[:,2] = np.pi/2
    return parms_sq.T 
    
def hexagonal(X):
    #Hexagonal a1 = a2, phi = 60 Label = 2
    parms_hex = np.zeros([X,3])
    parms_hex[:,0] = np.random.uniform(low = 0.8, high = 2.0,size = X)
    parms_hex[:,1] = parms_hex[:,0]
    parms_hex[:,2] = (np.pi/3)
    return parms_hex.T

def rectangular(X):    
    #rectangular a1 != a2, phi = 90 Label =3
    parms_rec = np.random.uniform(low = [0.8,0.8,(np.pi/2)], high = [2.0,2.2,(np.pi/2)],size = [X*2,3])
    diff_rec = np.abs(((parms_rec[:,0]-parms_rec[:,1])*100)/parms_rec[:,1])
    
    parms_rec = parms_rec[diff_rec > 20.0,:]
    parms_rec = parms_rec[0:X,:]
    return parms_rec.T

def centered(X):    
    #centered  a1 = a2, phi != 90, Label = 4
    bond_len = np.zeros([X,2])
    bond_len[:,0] = np.random.uniform(low = 0.8, high = 2.0,size = X)
    bond_len[:,1] = bond_len[:,0]
    phi = np.zeros((X,1))
    phi[0:int(0.7*X)] = np.random.uniform(low = [0.0], high = [((55.0/180)*np.pi)],size = [int(0.7*X),1])
    phi[int(0.7*X):X] = np.random.uniform(low = [((65.0/180)*np.pi)], high = [((90.0/180)*np.pi)],size = [int(.3*X),1])
    parms_cen = np.concatenate((bond_len,phi),axis = 1)
    return parms_cen.T
    
 
    

In [7]:
from scipy.ndimage import zoom



def clipped_zoom(img, zoom_factor, **kwargs):

    h, w = img.shape[:2]

    # For multichannel images we don't want to apply the zoom factor to the RGB
    # dimension, so instead we create a tuple of zoom factors, one per array
    # dimension, with 1's for any trailing dimensions after the width and height.
    zoom_tuple = (zoom_factor,) * 2 + (1,) * (img.ndim - 2)

    # Zooming out
    if zoom_factor < 1:

        # Bounding box of the zoomed-out image within the output array
        zh = int(np.round(h * zoom_factor))
        zw = int(np.round(w * zoom_factor))
        top = (h - zh) // 2
        left = (w - zw) // 2

        # Zero-padding
        out = np.zeros_like(img)
        out[top:top+zh, left:left+zw] = zoom(img, zoom_tuple, **kwargs)

    # Zooming in
    elif zoom_factor > 1:

        # Bounding box of the zoomed-in region within the input array
        zh = int(np.round(h / zoom_factor))
        zw = int(np.round(w / zoom_factor))
        top = (h - zh) // 2
        left = (w - zw) // 2

        out = zoom(img[top:top+zh, left:left+zw], zoom_tuple, **kwargs)

        # `out` might still be slightly larger than `img` due to rounding, so
        # trim off any extra pixels at the edges
        trim_top = ((out.shape[0] - h) // 2)
        trim_left = ((out.shape[1] - w) // 2)
        out = out[trim_top:trim_top+h, trim_left:trim_left+w]

    # If zoom_factor == 1, just return the input array
    else:
        out = img
    return out

In [10]:
def images_final(parms):
  atom_pos = lattice(parms)
  #atom_pos_dis = distortions(atom_pos, parms)
  image_atoms = atom_to_image(atom_pos)
  con_img = convolve_atomic_image(image_atoms)
  crop_img = crop(con_img)
  img_final = fft(crop_img)
  img_final = clipped_zoom(img_final,2)
 # img_final = crop(img_final)
  return img_final

In [11]:
dir_ = "new_data/"
dir = ["oblique","centered","square","rectangular","hexagonal"]
for i in dir:   
    params_test = locals()[i](100)
    params_train = locals()[i](1000)
    print(params_test.shape)
    params_val = locals()[i](100)
    z = 0
    final_train = images_final(params_train)
    for j in final_train:
        try:
            os.mkdir(dir_+"train/"+i)
        except:
            pass
        plt.imsave(dir_+"train/"+i+'/'+i+str(z)+".png",j,format = 'png')
        print(i+"~"+str(z))
        z += 1
    final_test = images_final(params_test)
    for j in final_test:
        try:
            os.mkdir(dir_+"test/"+i)
        except:
            pass        
        #final = images_final(j)
        plt.imsave(dir_+"test"+'/'+i+"/"+i+str(z)+".png",j,format = 'png')
        print(i+"~"+str(z))
        z += 1
    final_val = images_final(params_val)
    for j in final_val:
        try:
            os.mkdir(dir_+"val/"+i)
        except:
            pass
        plt.imsave(dir_+"val/"+i+'/'+str(z)+i+".png",j,format = 'png')
        print(i+"~"+str(z))
        z += 1

(3, 100)
1000
oblique~0
oblique~1
oblique~2
oblique~3
oblique~4
oblique~5
oblique~6
oblique~7
oblique~8
oblique~9
oblique~10
oblique~11
oblique~12
oblique~13
oblique~14
oblique~15
oblique~16
oblique~17
oblique~18
oblique~19
oblique~20
oblique~21
oblique~22
oblique~23
oblique~24
oblique~25
oblique~26
oblique~27
oblique~28
oblique~29
oblique~30
oblique~31
oblique~32
oblique~33
oblique~34
oblique~35
oblique~36
oblique~37
oblique~38
oblique~39
oblique~40
oblique~41
oblique~42
oblique~43
oblique~44
oblique~45
oblique~46
oblique~47
oblique~48
oblique~49
oblique~50
oblique~51
oblique~52
oblique~53
oblique~54
oblique~55
oblique~56
oblique~57
oblique~58
oblique~59
oblique~60
oblique~61
oblique~62
oblique~63
oblique~64
oblique~65
oblique~66
oblique~67
oblique~68
oblique~69
oblique~70
oblique~71
oblique~72
oblique~73
oblique~74
oblique~75
oblique~76
oblique~77
oblique~78
oblique~79
oblique~80
oblique~81
oblique~82
oblique~83
oblique~84
oblique~85
oblique~86
oblique~87
oblique~88
oblique~89
obliqu

oblique~741
oblique~742
oblique~743
oblique~744
oblique~745
oblique~746
oblique~747
oblique~748
oblique~749
oblique~750
oblique~751
oblique~752
oblique~753
oblique~754
oblique~755
oblique~756
oblique~757
oblique~758
oblique~759
oblique~760
oblique~761
oblique~762
oblique~763
oblique~764
oblique~765
oblique~766
oblique~767
oblique~768
oblique~769
oblique~770
oblique~771
oblique~772
oblique~773
oblique~774
oblique~775
oblique~776
oblique~777
oblique~778
oblique~779
oblique~780
oblique~781
oblique~782
oblique~783
oblique~784
oblique~785
oblique~786
oblique~787
oblique~788
oblique~789
oblique~790
oblique~791
oblique~792
oblique~793
oblique~794
oblique~795
oblique~796
oblique~797
oblique~798
oblique~799
oblique~800
oblique~801
oblique~802
oblique~803
oblique~804
oblique~805
oblique~806
oblique~807
oblique~808
oblique~809
oblique~810
oblique~811
oblique~812
oblique~813
oblique~814
oblique~815
oblique~816
oblique~817
oblique~818
oblique~819
oblique~820
oblique~821
oblique~822
oblique~823
obli

centered~218
centered~219
centered~220
centered~221
centered~222
centered~223
centered~224
centered~225
centered~226
centered~227
centered~228
centered~229
centered~230
centered~231
centered~232
centered~233
centered~234
centered~235
centered~236
centered~237
centered~238
centered~239
centered~240
centered~241
centered~242
centered~243
centered~244
centered~245
centered~246
centered~247
centered~248
centered~249
centered~250
centered~251
centered~252
centered~253
centered~254
centered~255
centered~256
centered~257
centered~258
centered~259
centered~260
centered~261
centered~262
centered~263
centered~264
centered~265
centered~266
centered~267
centered~268
centered~269
centered~270
centered~271
centered~272
centered~273
centered~274
centered~275
centered~276
centered~277
centered~278
centered~279
centered~280
centered~281
centered~282
centered~283
centered~284
centered~285
centered~286
centered~287
centered~288
centered~289
centered~290
centered~291
centered~292
centered~293
centered~294

centered~880
centered~881
centered~882
centered~883
centered~884
centered~885
centered~886
centered~887
centered~888
centered~889
centered~890
centered~891
centered~892
centered~893
centered~894
centered~895
centered~896
centered~897
centered~898
centered~899
centered~900
centered~901
centered~902
centered~903
centered~904
centered~905
centered~906
centered~907
centered~908
centered~909
centered~910
centered~911
centered~912
centered~913
centered~914
centered~915
centered~916
centered~917
centered~918
centered~919
centered~920
centered~921
centered~922
centered~923
centered~924
centered~925
centered~926
centered~927
centered~928
centered~929
centered~930
centered~931
centered~932
centered~933
centered~934
centered~935
centered~936
centered~937
centered~938
centered~939
centered~940
centered~941
centered~942
centered~943
centered~944
centered~945
centered~946
centered~947
centered~948
centered~949
centered~950
centered~951
centered~952
centered~953
centered~954
centered~955
centered~956

square~383
square~384
square~385
square~386
square~387
square~388
square~389
square~390
square~391
square~392
square~393
square~394
square~395
square~396
square~397
square~398
square~399
square~400
square~401
square~402
square~403
square~404
square~405
square~406
square~407
square~408
square~409
square~410
square~411
square~412
square~413
square~414
square~415
square~416
square~417
square~418
square~419
square~420
square~421
square~422
square~423
square~424
square~425
square~426
square~427
square~428
square~429
square~430
square~431
square~432
square~433
square~434
square~435
square~436
square~437
square~438
square~439
square~440
square~441
square~442
square~443
square~444
square~445
square~446
square~447
square~448
square~449
square~450
square~451
square~452
square~453
square~454
square~455
square~456
square~457
square~458
square~459
square~460
square~461
square~462
square~463
square~464
square~465
square~466
square~467
square~468
square~469
square~470
square~471
square~472
square~473

square~1153
square~1154
square~1155
square~1156
square~1157
square~1158
square~1159
square~1160
square~1161
square~1162
square~1163
square~1164
square~1165
square~1166
square~1167
square~1168
square~1169
square~1170
square~1171
square~1172
square~1173
square~1174
square~1175
square~1176
square~1177
square~1178
square~1179
square~1180
square~1181
square~1182
square~1183
square~1184
square~1185
square~1186
square~1187
square~1188
square~1189
square~1190
square~1191
square~1192
square~1193
square~1194
square~1195
square~1196
square~1197
square~1198
square~1199
(3, 100)
1000
rectangular~0
rectangular~1
rectangular~2
rectangular~3
rectangular~4
rectangular~5
rectangular~6
rectangular~7
rectangular~8
rectangular~9
rectangular~10
rectangular~11
rectangular~12
rectangular~13
rectangular~14
rectangular~15
rectangular~16
rectangular~17
rectangular~18
rectangular~19
rectangular~20
rectangular~21
rectangular~22
rectangular~23
rectangular~24
rectangular~25
rectangular~26
rectangular~27
rectangular~

rectangular~493
rectangular~494
rectangular~495
rectangular~496
rectangular~497
rectangular~498
rectangular~499
rectangular~500
rectangular~501
rectangular~502
rectangular~503
rectangular~504
rectangular~505
rectangular~506
rectangular~507
rectangular~508
rectangular~509
rectangular~510
rectangular~511
rectangular~512
rectangular~513
rectangular~514
rectangular~515
rectangular~516
rectangular~517
rectangular~518
rectangular~519
rectangular~520
rectangular~521
rectangular~522
rectangular~523
rectangular~524
rectangular~525
rectangular~526
rectangular~527
rectangular~528
rectangular~529
rectangular~530
rectangular~531
rectangular~532
rectangular~533
rectangular~534
rectangular~535
rectangular~536
rectangular~537
rectangular~538
rectangular~539
rectangular~540
rectangular~541
rectangular~542
rectangular~543
rectangular~544
rectangular~545
rectangular~546
rectangular~547
rectangular~548
rectangular~549
rectangular~550
rectangular~551
rectangular~552
rectangular~553
rectangular~554
rectangu

rectangular~1055
rectangular~1056
rectangular~1057
rectangular~1058
rectangular~1059
rectangular~1060
rectangular~1061
rectangular~1062
rectangular~1063
rectangular~1064
rectangular~1065
rectangular~1066
rectangular~1067
rectangular~1068
rectangular~1069
rectangular~1070
rectangular~1071
rectangular~1072
rectangular~1073
rectangular~1074
rectangular~1075
rectangular~1076
rectangular~1077
rectangular~1078
rectangular~1079
rectangular~1080
rectangular~1081
rectangular~1082
rectangular~1083
rectangular~1084
rectangular~1085
rectangular~1086
rectangular~1087
rectangular~1088
rectangular~1089
rectangular~1090
rectangular~1091
rectangular~1092
rectangular~1093
rectangular~1094
rectangular~1095
rectangular~1096
rectangular~1097
rectangular~1098
rectangular~1099
100
rectangular~1100
rectangular~1101
rectangular~1102
rectangular~1103
rectangular~1104
rectangular~1105
rectangular~1106
rectangular~1107
rectangular~1108
rectangular~1109
rectangular~1110
rectangular~1111
rectangular~1112
rectangula

hexagonal~426
hexagonal~427
hexagonal~428
hexagonal~429
hexagonal~430
hexagonal~431
hexagonal~432
hexagonal~433
hexagonal~434
hexagonal~435
hexagonal~436
hexagonal~437
hexagonal~438
hexagonal~439
hexagonal~440
hexagonal~441
hexagonal~442
hexagonal~443
hexagonal~444
hexagonal~445
hexagonal~446
hexagonal~447
hexagonal~448
hexagonal~449
hexagonal~450
hexagonal~451
hexagonal~452
hexagonal~453
hexagonal~454
hexagonal~455
hexagonal~456
hexagonal~457
hexagonal~458
hexagonal~459
hexagonal~460
hexagonal~461
hexagonal~462
hexagonal~463
hexagonal~464
hexagonal~465
hexagonal~466
hexagonal~467
hexagonal~468
hexagonal~469
hexagonal~470
hexagonal~471
hexagonal~472
hexagonal~473
hexagonal~474
hexagonal~475
hexagonal~476
hexagonal~477
hexagonal~478
hexagonal~479
hexagonal~480
hexagonal~481
hexagonal~482
hexagonal~483
hexagonal~484
hexagonal~485
hexagonal~486
hexagonal~487
hexagonal~488
hexagonal~489
hexagonal~490
hexagonal~491
hexagonal~492
hexagonal~493
hexagonal~494
hexagonal~495
hexagonal~496
hexago

hexagonal~1060
hexagonal~1061
hexagonal~1062
hexagonal~1063
hexagonal~1064
hexagonal~1065
hexagonal~1066
hexagonal~1067
hexagonal~1068
hexagonal~1069
hexagonal~1070
hexagonal~1071
hexagonal~1072
hexagonal~1073
hexagonal~1074
hexagonal~1075
hexagonal~1076
hexagonal~1077
hexagonal~1078
hexagonal~1079
hexagonal~1080
hexagonal~1081
hexagonal~1082
hexagonal~1083
hexagonal~1084
hexagonal~1085
hexagonal~1086
hexagonal~1087
hexagonal~1088
hexagonal~1089
hexagonal~1090
hexagonal~1091
hexagonal~1092
hexagonal~1093
hexagonal~1094
hexagonal~1095
hexagonal~1096
hexagonal~1097
hexagonal~1098
hexagonal~1099
100
hexagonal~1100
hexagonal~1101
hexagonal~1102
hexagonal~1103
hexagonal~1104
hexagonal~1105
hexagonal~1106
hexagonal~1107
hexagonal~1108
hexagonal~1109
hexagonal~1110
hexagonal~1111
hexagonal~1112
hexagonal~1113
hexagonal~1114
hexagonal~1115
hexagonal~1116
hexagonal~1117
hexagonal~1118
hexagonal~1119
hexagonal~1120
hexagonal~1121
hexagonal~1122
hexagonal~1123
hexagonal~1124
hexagonal~1125
hexago

In [34]:
params_train.shape

(3, 1000)

In [37]:
params_val.shape

(3, 400)